In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

from spacy.tokens import DocBin

from tqdm import tqdm

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

In [3]:
new_train = train_data[(train_data.feature_num == 1)][['pn_num','annotation',"location","feature_num"]]
new_train

,pn_num,annotation,location,feature_num
1,16,"['mom with ""thyroid disease']",['668 693'],1
14,41,['MOM HAS THYROID PROBLEMS'],['532 556'],1
27,46,['mom: thyroid issues'],['803 822'],1
40,82,['Mom thyroid problem'],['633 652'],1
53,100,['Mom-thyroid problem'],['714 733'],1
...,...,...,...,...
1236,2315,['mother thyroid condition'],['720 726;731 748'],1
1249,2425,['Mother hx of thyroid disease'],['594 622'],1
1262,2428,['mother hyroid problem'],['551 557;566 580'],1
1275,2430,['thyroid dx mother'],['814 824;832 838'],1


In [4]:
new_train = new_train[new_train.annotation != '[]']
new_train

,pn_num,annotation,location,feature_num
1,16,"['mom with ""thyroid disease']",['668 693'],1
14,41,['MOM HAS THYROID PROBLEMS'],['532 556'],1
27,46,['mom: thyroid issues'],['803 822'],1
40,82,['Mom thyroid problem'],['633 652'],1
53,100,['Mom-thyroid problem'],['714 733'],1
...,...,...,...,...
1236,2315,['mother thyroid condition'],['720 726;731 748'],1
1249,2425,['Mother hx of thyroid disease'],['594 622'],1
1262,2428,['mother hyroid problem'],['551 557;566 580'],1
1275,2430,['thyroid dx mother'],['814 824;832 838'],1


In [5]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train

,pn_num,annotation,location,feature_num
1,16,"'mom with ""thyroid disease'",'668 693',1
14,41,'MOM HAS THYROID PROBLEMS','532 556',1
27,46,'mom: thyroid issues','803 822',1
40,82,'Mom thyroid problem','633 652',1
53,100,'Mom-thyroid problem','714 733',1
...,...,...,...,...
1236,2315,'mother thyroid condition','720 726;731 748',1
1249,2425,'Mother hx of thyroid disease','594 622',1
1262,2428,'mother hyroid problem','551 557;566 580',1
1275,2430,'thyroid dx mother','814 824;832 838',1


In [6]:
new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))
new_train

,pn_num,annotation,location,feature_num
0,16,"'mom with ""thyroid disease'",'668 693',1
1,41,'MOM HAS THYROID PROBLEMS','532 556',1
2,46,'mom: thyroid issues','803 822',1
3,82,'Mom thyroid problem','633 652',1
4,100,'Mom-thyroid problem','714 733',1
...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1
88,2425,'Mother hx of thyroid disease','594 622',1
89,2428,'mother hyroid problem','551 557;566 580',1
90,2430,'thyroid dx mother','814 824;832 838',1


In [7]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])
new_train

,pn_num,annotation,location,feature_num,start_location,end_location
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556
2,46,'mom: thyroid issues','803 822',1,803,822
3,82,'Mom thyroid problem','633 652',1,633,652
4,100,'Mom-thyroid problem','714 733',1,714,733
...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748
88,2425,'Mother hx of thyroid disease','594 622',1,594,622
89,2428,'mother hyroid problem','551 557;566 580',1,551,580
90,2430,'thyroid dx mother','814 824;832 838',1,814,838


In [8]:
new_train = new_train.merge(patient_notes, on='pn_num', how='left')
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...
...,...,...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...
90,2430,'thyroid dx mother','814 824;832 838',1,814,838,0,Mr. Cleveland is a 17 yo male who presents wit...


In [9]:
new_train["New_annotation"] = new_train.apply(lambda x: x.pn_history[int(x.start_location):int(x.end_location)],axis=1)

In [10]:
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[']''','',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: x.strip())
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...,mom: thyroid issues
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem
...,...,...,...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...,mother has thyroid condition
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem
90,2430,'thyroid dx mother','814 824;832 838',1,814,838,0,Mr. Cleveland is a 17 yo male who presents wit...,thyroid dx in his mother


In [11]:
new_train["New_pn_history"] = new_train["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_pn_history"] = new_train["New_pn_history"].apply(lambda x: re.sub(r'''[']''','',x))
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI: 17yo M presents with palpitations. Patien...
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...,mom: thyroid issues,Mr. Cleveland is a 17yo M who was consented by...
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem,17 yo M w no cardiac or arrhythmia PMH presen...
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem,HPI: Dillon Cleveland is an otherwise healthy ...
...,...,...,...,...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...,mother has thyroid condition,17 yo M who presents with episodes of heart pa...
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease,17 yo CC palpitation -Started 3 months ago ...
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem,dillon cleveland is a 17yo male with no PMH th...
90,2430,'thyroid dx mother','814 824;832 838',1,814,838,0,Mr. Cleveland is a 17 yo male who presents wit...,thyroid dx in his mother,Mr. Cleveland is a 17 yo male who presents wit...


In [12]:
def word_lemm(sentence):
    word_list = nltk.word_tokenize(sentence)
    
    lemmaztier = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmaztier.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [13]:
new_train["New_annotation"] = new_train["New_annotation"].apply(word_lemm)
new_train["New_pn_history"] = new_train["New_pn_history"].apply(word_lemm)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with palpitation . Patien...
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...,mom : thyroid issue,Mr. Cleveland is a 17yo M who wa consented by ...
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem,17 yo M w no cardiac or arrhythmia PMH present...
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem,HPI : Dillon Cleveland is an otherwise healthy...
...,...,...,...,...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...,mother ha thyroid condition,17 yo M who present with episode of heart palp...
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease,17 yo CC palpitation -Started 3 month ago ha e...
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem,dillon cleveland is a 17yo male with no PMH th...
90,2430,'thyroid dx mother','814 824;832 838',1,814,838,0,Mr. Cleveland is a 17 yo male who presents wit...,thyroid dx in his mother,Mr. Cleveland is a 17 yo male who present with...


In [14]:
new_train["new_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(638, 662), match='mom ..."
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,"<re.Match object; span=(529, 553), match='MOM ..."
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...,mom : thyroid issue,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(777, 796), match='mom ..."
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(608, 627), match='Mom ..."
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(681, 700), match='Mom-..."
...,...,...,...,...,...,...,...,...,...,...,...
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...,mother ha thyroid condition,17 yo M who present with episode of heart palp...,"<re.Match object; span=(698, 725), match='moth..."
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease,17 yo CC palpitation -Started 3 month ago ha e...,"<re.Match object; span=(559, 587), match='Moth..."
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(535, 564), match='moth..."
90,2430,'thyroid dx mother','814 824;832 838',1,814,838,0,Mr. Cleveland is a 17 yo male who presents wit...,thyroid dx in his mother,Mr. Cleveland is a 17 yo male who present with...,"<re.Match object; span=(792, 816), match='thyr..."


In [15]:
drop_index = []
for  index,row in new_train.iterrows():
    if row["new_location"] == None:
        drop_index.append(index)
print(drop_index)

[91]


In [16]:
new_train = new_train.drop(drop_index,axis = 0)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(638, 662), match='mom ..."
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,532,556,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,"<re.Match object; span=(529, 553), match='MOM ..."
2,46,'mom: thyroid issues','803 822',1,803,822,0,Mr. Cleveland is a 17yo M who was consented by...,mom : thyroid issue,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(777, 796), match='mom ..."
3,82,'Mom thyroid problem','633 652',1,633,652,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(608, 627), match='Mom ..."
4,100,'Mom-thyroid problem','714 733',1,714,733,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(681, 700), match='Mom-..."
...,...,...,...,...,...,...,...,...,...,...,...
86,2308,'Mom has thyroid dz','928 946',1,928,946,0,17 yo male with no significant PMH presenting ...,Mom ha thyroid dz,17 yo male with no significant PMH presenting ...,"<re.Match object; span=(898, 915), match='Mom ..."
87,2315,'mother thyroid condition','720 726;731 748',1,720,748,0,17 yo M who presents with episodes of heart pa...,mother ha thyroid condition,17 yo M who present with episode of heart palp...,"<re.Match object; span=(698, 725), match='moth..."
88,2425,'Mother hx of thyroid disease','594 622',1,594,622,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease,17 yo CC palpitation -Started 3 month ago ha e...,"<re.Match object; span=(559, 587), match='Moth..."
89,2428,'mother hyroid problem','551 557;566 580',1,551,580,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(535, 564), match='moth..."


In [17]:
new_train["start_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).start(),axis =1)
new_train["end_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).end(),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,"'mom with ""thyroid disease'",'668 693',1,638,662,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(638, 662), match='mom ..."
1,41,'MOM HAS THYROID PROBLEMS','532 556',1,529,553,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,MOM HAS THYROID PROBLEMS,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,"<re.Match object; span=(529, 553), match='MOM ..."
2,46,'mom: thyroid issues','803 822',1,777,796,0,Mr. Cleveland is a 17yo M who was consented by...,mom : thyroid issue,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(777, 796), match='mom ..."
3,82,'Mom thyroid problem','633 652',1,608,627,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Mom thyroid problem,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(608, 627), match='Mom ..."
4,100,'Mom-thyroid problem','714 733',1,681,700,0,HPI: Dillon Cleveland is an otherwise healthy ...,Mom-thyroid problem,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(681, 700), match='Mom-..."
...,...,...,...,...,...,...,...,...,...,...,...
86,2308,'Mom has thyroid dz','928 946',1,898,915,0,17 yo male with no significant PMH presenting ...,Mom ha thyroid dz,17 yo male with no significant PMH presenting ...,"<re.Match object; span=(898, 915), match='Mom ..."
87,2315,'mother thyroid condition','720 726;731 748',1,698,725,0,17 yo M who presents with episodes of heart pa...,mother ha thyroid condition,17 yo M who present with episode of heart palp...,"<re.Match object; span=(698, 725), match='moth..."
88,2425,'Mother hx of thyroid disease','594 622',1,559,587,0,17 yo CC palpitation \r\n-Started 3 months ago...,Mother hx of thyroid disease,17 yo CC palpitation -Started 3 month ago ha e...,"<re.Match object; span=(559, 587), match='Moth..."
89,2428,'mother hyroid problem','551 557;566 580',1,535,564,0,dillon cleveland is a 17yo male with no PMH th...,mother with a thyroid problem,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(535, 564), match='moth..."


In [18]:
sep_combined_case0  = new_train.copy()

In [19]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in sep_combined_case0.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            
            if row["start_location"] in start_location:
                if row["end_location"] >= sep_combined_case0.iloc[index -1 ,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= sep_combined_case0.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])            

In [20]:
print(drop_index), len(drop_index)

[7]


(None, 1)

In [21]:
sep_combined_case0 = sep_combined_case0.drop(index=drop_index)

In [22]:
pn_Num_list = list(sep_combined_case0.pn_num.unique())

In [23]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in sep_combined_case0[sep_combined_case0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = sep_combined_case0[sep_combined_case0.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list

In [24]:
final_train[6]

('17 yo M college student come to the clinic due to heart pounding . He state the he ha had episode of heart pounding for the last 3-4 month . Nothing make it better or worse and it go away on it own . he denies any chest pain but doe report chest pressure during some of the episode . He denies warmth or sweating recent illness abdominal pain or N V tingling in extremity recent illness or trauma . He is new to college and ha been taking aderol a few time a week . The last time he took aderol wa 2 day ago which wa also the last time he had episode of palpitation . ROS : negative except for above PMH PSH : none medication : aderol self administered . knda FH : Mother thyroid disease father heart attach at 52 is okay now . SH : no change in weight or appetite he is on the meal plan and try to be healthy he play intermural soccer and run no smoking alcohol hx . Triend marijuanna us aderol . Stress college',
 {'entities': [(665, 687, 'Feature_1')]})

In [25]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [26]:
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [27]:
nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [28]:
train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

len(train),len(validation)

(71, 18)

In [29]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 71/71 [00:00<00:00, 865.04it/s]


In [30]:
# Validation data set
camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 897.12it/s]


In [31]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-05 23:49:33.259864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:49:33.259904: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [32]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     61.83    0.00    0.00    0.00    0.00
  2     200        104.32   1317.80   75.68   73.68   77.78    0.76
  5     400         23.13     35.46   80.00   82.35   77.78    0.80
  8     600          3.88      7.78   77.78   77.78   77.78    0.78
 11     800          0.00      0.00   75.68   73.68   77.78    0.76
 14    1000          0.00      0.00   83.33   83.33   83.33    0.83
 16    1200          0.00      0.00   83.33   83.33   83.33    0.83
 19    1400          0.00      0.00   83.33   83.33 

2022-04-05 23:49:36.837244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:49:36.837289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-05 23:49:39,141] [INFO] Set up nlp object from config
[2022-04-05 23:49:39,147] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-05 23:49:39,150] [INFO] Created vocabulary
[2022-04-05 23:49:39,150] [INFO] Finished initializing nlp object
[2022-04-05 23:49:39,505] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
